In [1]:
%matplotlib inline

import pickle
import seaborn as sns
import copy
import time
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix,roc_curve, auc

In [2]:
def classification_report_csv(report, methodName):
    lines = report.split('\n')
    line1 = lines[2]
    line2 = lines[3]
    line3 = lines[5]
    line4 = lines[6]
    line5 = lines[7]
    
    row = {}
    
    row_data1 = ' '.join(line1.split())   
    row_data1 = row_data1.split(' ')
    row_data2 = ' '.join(line2.split())   
    row_data2 = row_data2.split(' ')
    row_data3 = ' '.join(line3.split())   
    row_data3 = row_data3.split(' ')
    row_data4 = ' '.join(line4.split())   
    row_data4 = row_data4.split(' ')
    row_data5 = ' '.join(line5.split())   
    row_data5 = row_data5.split(' ')
    
    row['Method'] = methodName
    row['Test Size'] = sizeOfTest
    row[f'precision class {row_data1[0]}'] = float(row_data1[1])    
    row[f'precision class {row_data2[0]}'] = float(row_data2[1])
    row[f'recall class {row_data1[0]}'] = float(row_data1[2])
    row[f'recall class {row_data2[0]}'] = float(row_data2[2])
    row[f'f1_score class {row_data1[0]}'] = float(row_data1[3])
    row[f'f1_score class {row_data2[0]}'] = float(row_data2[3])
    row[f'support class {row_data1[0]}'] = float(row_data1[4])
    row[f'support class {row_data2[0]}'] = float(row_data2[4])
    row['Accuracy f1-score'] = float(row_data3[1])
    row['Accuracy support'] = float(row_data3[2])
    row['macro avg precision'] = float(row_data4[2])
    row['macro avg recall'] = float(row_data4[3])
    row['macro avg f1score'] = float(row_data4[4])
    row['macro avg support'] = float(row_data4[5])
    row['weighted avg precision'] = float(row_data5[2])
    row['weighted avg recall'] = float(row_data5[3])
    row['weighted avg f1score'] = float(row_data5[4])
    row['weighted avg support'] = float(row_data5[5])
    
    report_data.append(row) 
    



In [3]:
df = pd.read_csv('final_dataset.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,...,In_degree_Cen_Src,Out_degree_Cen_Src,In_degree_Cen_Dst,Out_degree_Cen_Dst,Eigenvec_Cen_Src,Eigenvec_Cen_Dst,Core_Num_Src,Core_Num_Dst,LCC_Source,LCC_Destination
0,6194,255,1,0.000016,0.000013,-1,0,4,0,5,...,0.0,0.000101,0.000000,0.000126,3.257143e-19,3.257143e-19,3,4,0.0,0.400000
1,6194,980,1,0.000016,0.000015,-1,0,4,0,15,...,0.0,0.000101,0.000000,0.000379,3.257143e-19,3.257143e-19,3,8,0.0,0.152381
2,6194,2992,1,0.000016,0.000016,4,0,4,3,0,...,0.0,0.000101,0.000076,0.000000,3.257143e-19,5.715021e-05,3,3,0.0,0.000000
3,6194,2507,1,0.000016,0.000057,3,0,4,8,14,...,0.0,0.000101,0.000202,0.000354,3.257143e-19,1.470102e-06,3,11,0.0,0.051948
4,6194,986,1,0.000016,0.000023,9,0,4,1,21,...,0.0,0.000101,0.000025,0.000531,3.257143e-19,7.491428e-18,3,11,0.0,0.225108


In [4]:
#PCA

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
scaled_data = scaler.transform(df)

In [5]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
pca.fit(scaled_data)

PCA(n_components=2)

In [6]:
x_pca = pca.transform(scaled_data)
print(scaled_data.shape)
print(x_pca.shape)

(70648, 22)
(70648, 2)


In [7]:
df.to_csv('preprocessed_data.csv')

In [8]:
g = nx.from_pandas_edgelist(df[['Source','Destination']], source='Source', target='Destination',create_using=nx.DiGraph())

In [9]:
df_x = df.drop(columns=['Source', 'Destination', 'Class'])
df_y = df[['Class']]

In [10]:
report_data = []
sizeOfTest = 0.3
# x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = sizeOfTest, random_state=10)

In [11]:
#Logistic Regression
for i in range(1,11):
    x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = sizeOfTest, random_state=10*i)
    start = time.time()
    lr = LogisticRegression(C=2,solver='liblinear', random_state=0).fit(x_train, y_train)
    end = time.time()
    print('Time taken to train model : ', round(end-start,2) , ' seconds')
    report = classification_report(lr.predict(x_test), y_test)
    classification_report_csv(report,"LR")

Time taken to train model :  0.48  seconds
Time taken to train model :  0.49  seconds
Time taken to train model :  0.4  seconds
Time taken to train model :  0.39  seconds
Time taken to train model :  0.43  seconds
Time taken to train model :  0.34  seconds
Time taken to train model :  0.38  seconds
Time taken to train model :  0.36  seconds
Time taken to train model :  0.35  seconds
Time taken to train model :  0.25  seconds


In [12]:
print(classification_report(lr.predict(x_test), y_test))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96     10937
           1       0.95      0.97      0.96     10258

    accuracy                           0.96     21195
   macro avg       0.96      0.96      0.96     21195
weighted avg       0.96      0.96      0.96     21195



In [13]:
#Random Forest

In [14]:
from sklearn.ensemble import RandomForestClassifier

for i in range(1,11):
    x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = sizeOfTest, random_state=10*i)
    start = time.time()
    rf=RandomForestClassifier(n_estimators=50,max_depth=9)

    #Train the model using the training sets y_pred=clf.predict(X_test)
    rf.fit(x_train,y_train)
    end = time.time()
    print('Time taken to train model : ', round(end-start,2) , ' seconds')
    y_pred=rf.predict(x_test)
    report = classification_report(y_pred, y_test)
    classification_report_csv(report,"RFC")



Time taken to train model :  1.43  seconds
Time taken to train model :  1.29  seconds
Time taken to train model :  1.49  seconds
Time taken to train model :  1.3  seconds
Time taken to train model :  1.33  seconds
Time taken to train model :  1.28  seconds
Time taken to train model :  1.37  seconds
Time taken to train model :  1.43  seconds
Time taken to train model :  1.39  seconds
Time taken to train model :  1.48  seconds


In [15]:
y_pred=rf.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97     10901
           1       0.96      0.97      0.97     10294

    accuracy                           0.97     21195
   macro avg       0.97      0.97      0.97     21195
weighted avg       0.97      0.97      0.97     21195



In [16]:
#Support Vector Machine

In [17]:
from sklearn import svm

for i in range(1,11):
    x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = sizeOfTest, random_state=10*i)
    start = time.time()
    #Create a svm Classifier
    sv = svm.SVC(kernel='linear') # Linear Kernel
    #Train the model using the training sets
    sv.fit(x_train, y_train)
    end = time.time()
    print('Time taken to train model : ', round(end-start,2) , ' seconds')
    y_pred=sv.predict(x_test)
    report = classification_report(y_pred, y_test)
    classification_report_csv(report,"SVM")



Time taken to train model :  50.63  seconds
Time taken to train model :  55.39  seconds
Time taken to train model :  60.77  seconds
Time taken to train model :  59.9  seconds
Time taken to train model :  52.31  seconds
Time taken to train model :  54.96  seconds
Time taken to train model :  50.8  seconds
Time taken to train model :  53.88  seconds
Time taken to train model :  49.04  seconds
Time taken to train model :  48.82  seconds


In [18]:
y_pred=sv.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96     11033
           1       0.94      0.97      0.96     10162

    accuracy                           0.96     21195
   macro avg       0.96      0.96      0.96     21195
weighted avg       0.96      0.96      0.96     21195



In [19]:
# XGBoost

In [20]:
for i in range(1,11):
    x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = sizeOfTest, random_state=10*i)
    start = time.time()
    bst = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1 , colsample_bytree = 0.7 ,objective='binary:logistic')

    #Train the model using the training sets y_pred=clf.predict(X_test)
    bst.fit(x_train,y_train)
    end = time.time()
    print('Time taken to train model : ', round(end-start,2) , ' seconds')
    y_pred=bst.predict(x_test)
    report = classification_report(y_pred, y_test)
    classification_report_csv(report,"XGBoost")

Time taken to train model :  1.21  seconds
Time taken to train model :  1.2  seconds
Time taken to train model :  1.22  seconds
Time taken to train model :  1.21  seconds
Time taken to train model :  1.22  seconds
Time taken to train model :  1.22  seconds
Time taken to train model :  1.22  seconds
Time taken to train model :  1.23  seconds
Time taken to train model :  1.25  seconds
Time taken to train model :  1.22  seconds


In [21]:
#Adaboost:

In [22]:
for i in range(1,11):
    x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = sizeOfTest, random_state=10*i)
    start = time.time()
    
    ada = AdaBoostClassifier(n_estimators=140,
                            learning_rate=1,
                            algorithm='SAMME.R',
                            random_state=1)
    ada.fit(x_train, y_train)
    
    end = time.time()
    
    print('Time taken to train model : ', round(end-start,2) , ' seconds')
    y_pred=ada.predict(x_test)
    report = classification_report(y_pred, y_test)
    
    classification_report_csv(report,"ADABoost")

Time taken to train model :  4.75  seconds
Time taken to train model :  4.7  seconds
Time taken to train model :  4.68  seconds
Time taken to train model :  4.76  seconds
Time taken to train model :  4.78  seconds
Time taken to train model :  4.71  seconds
Time taken to train model :  4.67  seconds
Time taken to train model :  4.8  seconds
Time taken to train model :  4.66  seconds
Time taken to train model :  4.69  seconds


In [24]:
dataframe = pd.DataFrame.from_dict(report_data)
dataframe.to_csv('classification_report.csv', index = False)